In [1]:
import os
cwd = os.getcwd()
print(cwd)
if cwd[-9:] == 'NoteBooks' or cwd[-5:] in ['/ilc2', '/data']:
    try:
        # Move up to root folder
        os.chdir(".." + ('/..' if cwd[-5:] == '/ilc2' else ''))
        print(f"Directory changed: {os.getcwd()}")
    except OSError:
        print("Can't change the Current Working Directory") 
else:
    print("No need to change the Current Working Directory")

/Users/romainfouilland/code/cours/P3A/data/ilc2
Directory changed: /Users/romainfouilland/code/cours/P3A


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from glob import glob

In [4]:
import json
import os
from pathlib import Path

import pandas as pd
import numpy as np

from compute_f import split_ts_seq, compute_step_positions
from io_f import read_data_file
from visualize_f import visualize_trajectory, visualize_heatmap, save_figure_to_html

floor_data_dir = './data/ilc2/data/site1/F1'
path_data_dir = floor_data_dir + '/path_data_files'
floor_plan_filename = floor_data_dir + '/floor_image.png'
floor_info_filename = floor_data_dir + '/floor_info.json'

save_dir = './data/ilc2/output/site1/F1'
path_image_save_dir = save_dir + '/path_images'
step_position_image_save_dir = save_dir
magn_image_save_dir = save_dir
wifi_image_save_dir = save_dir + '/wifi_images'
ibeacon_image_save_dir = save_dir + '/ibeacon_images'
wifi_count_image_save_dir = save_dir

In [5]:
Path(path_image_save_dir).mkdir(parents=True, exist_ok=True)
Path(magn_image_save_dir).mkdir(parents=True, exist_ok=True)
Path(wifi_image_save_dir).mkdir(parents=True, exist_ok=True)
Path(ibeacon_image_save_dir).mkdir(parents=True, exist_ok=True)

with open(floor_info_filename) as f:
    floor_info = json.load(f)
width_meter = floor_info["map_info"]["width"]
height_meter = floor_info["map_info"]["height"]

# Investigating a trace file

In [6]:
from tqdm.auto import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from integrations.ilc2 import get_input, json_points_to_array, collect_stats, get_filenames, collect_paths, visualize, load_stats, write_stats

In [7]:
path_filenames = get_filenames(path_data_dir)[:10]
path_filenames

[PosixPath('/Users/romainfouilland/code/cours/P3A/data/ilc2/data/site1/F1/path_data_files/5dd9fd5ec5b77e0006b173da.txt'),
 PosixPath('/Users/romainfouilland/code/cours/P3A/data/ilc2/data/site1/F1/path_data_files/5dd9fd649191710006b570f4.txt'),
 PosixPath('/Users/romainfouilland/code/cours/P3A/data/ilc2/data/site1/F1/path_data_files/5dd9fd4ec5b77e0006b173ce.txt'),
 PosixPath('/Users/romainfouilland/code/cours/P3A/data/ilc2/data/site1/F1/path_data_files/5dd9fd47c5b77e0006b173c8.txt'),
 PosixPath('/Users/romainfouilland/code/cours/P3A/data/ilc2/data/site1/F1/path_data_files/5dd9fd669191710006b570f6.txt'),
 PosixPath('/Users/romainfouilland/code/cours/P3A/data/ilc2/data/site1/F1/path_data_files/5dd9ef8dc5b77e0006b17359.txt'),
 PosixPath('/Users/romainfouilland/code/cours/P3A/data/ilc2/data/site1/F1/path_data_files/5dd9fd60c5b77e0006b173dc.txt'),
 PosixPath('/Users/romainfouilland/code/cours/P3A/data/ilc2/data/site1/F1/path_data_files/5dd9fd67c5b77e0006b173e4.txt'),
 PosixPath('/Users/romai

In [8]:
json_paths = collect_paths(path_filenames)
json_paths

  0%|          | 0/10 [00:00<?, ?it/s]

Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.


[{'file': '/Users/romainfouilland/code/cours/P3A/data/ilc2/data/site1/F1/path_data_files/5dd9fd5ec5b77e0006b173da.txt',
  't0': 0,
  'tf': -1574566385021.0,
  'points': [{'r_id': 1,
    'x': 196.26472,
    'y': 114.8901,
    't': 0,
    't_id': 0,
    'r_type': 'bluetooth',
    '_data': {'t': 0.0}},
   {'r_id': 2,
    'x': 196.94728,
    'y': 117.62149,
    't': 3755,
    't_id': 0,
    'r_type': 'bluetooth',
    '_data': {'t': 3755.0}},
   {'r_id': 3,
    'x': 208.12675,
    'y': 116.20255,
    't': 16911,
    't_id': 0,
    'r_type': 'bluetooth',
    '_data': {'t': 16911.0}},
   {'r_id': 4,
    'x': 206.11342,
    'y': 113.105,
    't': 22261,
    't_id': 0,
    'r_type': 'bluetooth',
    '_data': {'t': 22261.0}},
   {'r_id': 5,
    'x': 199.0444,
    'y': 109.97728,
    't': 31329,
    't_id': 0,
    'r_type': 'bluetooth',
    '_data': {'t': 31329.0},
    'md': {'exit': 1}}],
  '_data': {'t0': 1574566416350.0, 'tf': 31329}},
 {'file': '/Users/romainfouilland/code/cours/P3A/data/ilc2

In [9]:
# Computing stats
stats = collect_stats(json_paths)
stats

  0%|          | 0/10 [00:00<?, ?it/s]

,filepath,nb_points,timestep_dev,timestep_avg,speed_dev,site,floor,path_id
0,/Users/romainfouilland/code/cours/P3A/data/ilc...,5,0.463237,7832.250000,0.089398,site1,F1,5dd9fd5ec5b77e0006b173da
1,/Users/romainfouilland/code/cours/P3A/data/ilc...,3,0.470829,12718.000000,0.150494,site1,F1,5dd9fd649191710006b570f4
2,/Users/romainfouilland/code/cours/P3A/data/ilc...,12,0.441736,4194.818182,0.252844,site1,F1,5dd9fd4ec5b77e0006b173ce
3,/Users/romainfouilland/code/cours/P3A/data/ilc...,3,0.431057,8046.500000,0.008296,site1,F1,5dd9fd47c5b77e0006b173c8
4,/Users/romainfouilland/code/cours/P3A/data/ilc...,5,0.283175,7731.750000,0.432103,site1,F1,5dd9fd669191710006b570f6
5,/Users/romainfouilland/code/cours/P3A/data/ilc...,20,0.530575,5506.526316,0.358767,site1,F1,5dd9ef8dc5b77e0006b17359
6,/Users/romainfouilland/code/cours/P3A/data/ilc...,4,0.427371,7645.333333,0.083973,site1,F1,5dd9fd60c5b77e0006b173dc
7,/Users/romainfouilland/code/cours/P3A/data/ilc...,6,0.294774,6833.000000,0.326024,site1,F1,5dd9fd67c5b77e0006b173e4
8,/Users/romainfouilland/code/cours/P3A/data/ilc...,11,0.341685,8113.400000,0.248465,site1,F1,5dd9fd3a9191710006b570d2
9,/Users/romainfouilland/code/cours/P3A/data/ilc...,7,0.343712,4957.500000,0.374727,site1,F1,5dd9efa69191710006b5708c


In [10]:
FULL = True
WRITE_STATS = True

data_folder = "./data/ilc2/data/"
traces_folder = '/path_data_files'

if FULL:
    data_folder = "/Volumes/DD Romain/dataP3a/indoor-location-navigation/train"
    traces_folder = ''

stats_path = data_folder + '/stats.csv'

In [11]:
# Get path_filenames
# If write_stats -> collect them all
# Else, load write_stats and do filtering
if WRITE_STATS:
    path_filenames = []
    for floor in tqdm(glob(data_folder + '/*/*/')):
        path_filenames.extend(get_filenames(floor + traces_folder))
else:
    stats_df = load_stats(stats_path)
    # Do filtering on stats_df
    path_filenames = list(map(Path, stats_df['filepath']))

path_filenames

  0%|          | 0/981 [00:00<?, ?it/s]

[PosixPath('/Volumes/DD Romain/dataP3a/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730aa280850006f3d005.txt'),
 PosixPath('/Volumes/DD Romain/dataP3a/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730b1506f2000638fc29.txt'),
 PosixPath('/Volumes/DD Romain/dataP3a/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730ca280850006f3d007.txt'),
 PosixPath('/Volumes/DD Romain/dataP3a/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730e1506f2000638fc2b.txt'),
 PosixPath('/Volumes/DD Romain/dataP3a/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730f1506f2000638fc2d.txt'),
 PosixPath('/Volumes/DD Romain/dataP3a/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e157310a280850006f3d009.txt'),
 PosixPath('/Volumes/DD Romain/dataP3a/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e1573121506f2000638fc2f.txt'),
 PosixPath('/Volumes/DD Romain/dataP3a/indoor-location-navigation/train/5a05

In [ ]:
json_paths = collect_paths(path_filenames)

if WRITE_STATS:
    stats_dfs = []
    stats_dfs.append(collect_stats(json_paths))
    stats = pd.concat(stats_dfs)
    write_stats(stats, stats_path)

df = stats.sort_values(by='nb_points', ascending=False)
df.describe()

  0%|          | 0/26925 [00:00<?, ?it/s]

Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading pa

Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading path -> done.
Loading pa

In [ ]:
df.describe()

In [ ]:
df.iloc[:50]

In [ ]:
df.hist()

In [ ]:
df[df['nb_points'] > 15].hist()

In [ ]:
scores = pd.concat([
    df.groupby(['site', 'floor']).sum()['nb_points'],
    df.groupby(['site', 'floor'])[['timestep_avg', 'timestep_dev']].mean(),
    df.groupby(['site', 'floor']).size().rename('size')
], axis=1)
scores[scores['nb_points'] > 500].sort_values(
    'size', ascending=False
    # 'timestep_dev'
).head(50)

In [ ]:
filtered_df = df[(df['nb_points'] > 15) & (df['timestep_dev'] < 0.4) & (df['timestep_avg'] < 10000)]
filtered_df.hist()
filtered_df

In [ ]:
filtered_df.groupby(['site', 'floor']).size()

In [ ]:
filtered_df.groupby(['site', 'floor']).size().idxmax(), filtered_df.groupby(['site', 'floor']).size().max()

In [ ]:
# site, floor = 'site1', 'F2'
site, floor = filtered_df.groupby(['site', 'floor']).size().idxmax()
final_df = filtered_df[(filtered_df['site'] == site) & (filtered_df['floor'] == floor)]
final_df.hist()
final_df

In [ ]:
final_df['nb_points'].sum(), (final_df['nb_points'] * final_df['timestep_avg']).sum() // 1000 / 60

In [ ]:
if FULL:
    floor_plan_filename = f"/Volumes/DD Romain/dataP3a/indoor-location-navigation/metadata/{site}/{floor}/floor_image.png"
    floor_info_filename = f"/Volumes/DD Romain/dataP3a/indoor-location-navigation/metadata/{site}/{floor}/floor_info.json"
    print(floor_plan_filename)
else:
    floor_plan_filename = floor_data_dir + '/floor_image.png'
    floor_info_filename = floor_data_dir + '/floor_info.json'
    
with open(floor_info_filename) as f:
    floor_info = json.load(f)
width_meter = floor_info["map_info"]["width"]
height_meter = floor_info["map_info"]["height"]

# Building figs
figs = []
for _, row in tqdm(final_df.iterrows()):
    figs.append(visualize(
        row['filepath'], floor_plan_filename, width_meter, height_meter, 
        show=False
    ))

In [ ]:
@interact(
    i=widgets.IntSlider(min=0, max=len(figs) - 1, step=1, value=5)
)
def show_path(i):
    figs[i].show()

# Conclusion & question
## Pipeline
- We collect traces with ILC2 module **(done)**
- We convert them to a manageable format
    - 1. Convert them to .json path to be injected in the solution with the API integration **(done)**
    - 2. Convert them to .plt Geolife format (integrations/helper.py > to_plt) -> if we are willing to round all times to the closest second
- We compute stats
    - 1. Compute stats on them and store all of them in a stats file to be able to do easy filtering (just like Geolife with .plt -> stats -> filters -> loaded traces) -> can compute stats when we convert them and include them in the json -> better to build stats on json as this could be reused later (API json format) -> but OK to write them back in the json directly too **(computation done | storage left)**
    - 2. Directly leverage .plt filtering pipeline **(just implement filter functions)**
- We agregate traces
    - We need same site + floor + timestep_avg (to be used as dt unless we are willing to scale a bit -> change the speed to keep the same positions in the building)
    - Aggregate them to use the same t0 and same dt and max(tf)
    
## Data
- How should we decide on bluetooth / camera points and camera traces ? Traces are short so perhaps we can do only beginning and end as bluetooth and rest as camera points with no trace. But seems a bit hard.
- How to deal with non-periodic recording times?
- What do we want to prove with the experiments?
- Build an ICL2 object to collect initial data, build fig, stats, ... and also reconciliation function which will return the final API json data?